# Predictor de Salarios - Electiva II


In [10]:
# Instalación
!pip install -q gspread pandas numpy scikit-learn google-auth-oauthlib
print("✅ Dependencias instaladas")

✅ Dependencias instaladas


In [11]:
# Autenticación
from google.colab import auth
from google.auth import default
import gspread

print("🔐 Autenticando...")
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
print("✅ Autenticación exitosa")

🔐 Autenticando...
✅ Autenticación exitosa


In [12]:
# Cargar datos
import pandas as pd

SPREADSHEET_ID = '1rZdcjfJtz0Bg40XAYb8_6Z70Sf_fMrMb6LXeijdlodk'

spreadsheet = gc.open_by_key(SPREADSHEET_ID)
worksheets = spreadsheet.worksheets()

print(f"\n📑 Pestañas:")
for i, ws in enumerate(worksheets, 1):
    print(f"   {i}. {ws.title}")

opcion = '1'
worksheet = worksheets[int(opcion) - 1]

data = worksheet.get_all_values()
df = pd.DataFrame(data[1:], columns=data[0])
df = df.replace('', pd.NA).dropna(how='all').dropna(axis=1, how='all')

print(f"\n✅ Cargados: {len(df)} registros")
print(f"\n📋 Columnas: {list(df.columns)}")
display(df.head(3))


📑 Pestañas:
   1. Hoja 1

✅ Cargados: 1451 registros

📋 Columnas: ['Titulo', 'Pagina', 'Descripción breve', 'Ciudad', 'Salario', 'Modalidad']


,Titulo,Pagina,Descripción breve,Ciudad,Salario,Modalidad
0,Desarrollador Master (Java/Springboot),https://co.computrabajo.com/ofertas/1,Desarrollador backend con experiencia en Java ...,Bogotá,"$ 8.400.000,00 (Mensual)",Remoto
1,Analista de Datos (Python),https://co.computrabajo.com/ofertas/2,"Análisis de datos con Python, SQL y PowerBI",Medellín,"$ 6.000.000,00 (Mensual)",Presencial
2,Preventista / Vendedor TAT / Representante com...,https://co.computrabajo.com/ofertas/3,"Ventas TAT, atención al cliente, rutas, distri...",Bogotá,"$ 2.500.000,00 (Mensual)",Presencial


In [13]:
# Detectar columnas
def encontrar_columna(df, palabras, tipo):
    for col in df.columns:
        if any(p in col.lower() for p in palabras):
            return col
    print(f"\n⚠️ No encontré columna de {tipo}")
    print(f"Columnas: {list(df.columns)}")
    return input(f"¿Cuál es {tipo}?: ").strip()

COLUMN_TITULO = encontrar_columna(df, ['titulo', 'title', 'puesto'], 'TÍTULO')
COLUMN_DESCRIPCION = encontrar_columna(df, ['descripcion', 'description', 'desc'], 'DESCRIPCIÓN')
COLUMN_SALARIO = encontrar_columna(df, ['salario', 'salary', 'sueldo'], 'SALARIO')

print(f"\n✅ Columnas:")
print(f"   Título: '{COLUMN_TITULO}'")
print(f"   Descripción: '{COLUMN_DESCRIPCION}'")
print(f"   Salario: '{COLUMN_SALARIO}'")


✅ Columnas:
   Título: 'Titulo'
   Descripción: 'Descripción breve'
   Salario: 'Salario'


In [14]:
# ========================================
# LIMPIEZA AVANZADA DE SALARIOS
# ========================================

import numpy as np
import re

def limpiar_salario_avanzado(salary_str):
    """
    Limpia salarios con casos complejos:
    - '$ 2.000.000,00 (Mensual)'
    - '$1,5 a $2 millones'
    - '$1.000.001 - $1.500.000' (promedio)
    - 'Salario confidencial' (None)
    - 'A Convenir' (None)
    """

    # Convertir a string y limpiar
    s = str(salary_str).strip()

    if pd.isna(salary_str) or not s:
        return None

    s_lower = s.lower()

    # 1. Palabras de exclusión
    exclusiones = ['confidencial', 'convenir', 'definir', 'negociar', 'aplica']
    if any(palabra in s_lower for palabra in exclusiones):
        return None

    # 2. Limpiar texto extra (Mensual, Comisiones, etc.)
    s = re.sub(r'\(.*?\)', '', s)  # Quitar paréntesis
    s = re.sub(r'\+.*', '', s)      # Quitar "+" y lo que sigue
    s = s.split('Comisiones')[0]    # Quitar "Comisiones"

    # 3. Detectar rangos: "$1.000.001 - $1.500.000" o "$1,5 a $2 millones"
    # Rangos con guión o "a"
    if '-' in s or ' a ' in s_lower or ' y ' in s_lower:
        # Intentar extraer dos números
        numeros = []

        # Buscar patrones como "1,5" o "2" (millones)
        if 'millon' in s_lower:
            # Ejemplo: "$1,5 a $2 millones" → [1.5, 2] millones
            matches = re.findall(r'(\d+[,.]?\d*)', s)
            for m in matches:
                try:
                    num = float(m.replace(',', '.'))
                    numeros.append(num * 1_000_000)  # Convertir a millones
                except:
                    pass
        else:
            # Ejemplo: "$1.000.001 - $1.500.000"
            matches = re.findall(r'\d[\d.,]*', s)
            for m in matches:
                try:
                    # Limpiar formato
                    clean = m.replace('.', '').replace(',', '')
                    num = float(clean)
                    if num > 100:  # Es un salario real
                        numeros.append(num)
                except:
                    pass

        # Si encontramos 2 números, hacer promedio
        if len(numeros) >= 2:
            promedio = sum(numeros[:2]) / 2
            if 1_000_000 <= promedio <= 50_000_000:
                return promedio

    # 4. Detectar "millones": "$3,5 millones" → 3.500.000
    if 'millon' in s_lower:
        matches = re.findall(r'(\d+[,.]?\d*)', s)
        if matches:
            try:
                num = float(matches[0].replace(',', '.'))
                value = num * 1_000_000
                if 1_000_000 <= value <= 50_000_000:
                    return value
            except:
                pass

    # 5. Extracción normal de números
    matches = re.findall(r'\d[\d.,]*', s)
    if not matches:
        return None

    number_str = matches[0].strip()

    try:
        # Detectar formato
        if ',' in number_str:
            # Formato: "2.000.000,00" (colombiano) o "2,000,000" (internacional)
            if '.' in number_str and number_str.rfind('.') > number_str.rfind(','):
                # "2,000,000.00" → quitar comas
                number_str = number_str.replace(',', '')
            else:
                # "2.000.000,00" → quitar puntos, coma → punto
                number_str = number_str.replace('.', '').replace(',', '.')
        else:
            # Solo puntos: "2.000.000" → quitar puntos
            number_str = number_str.replace('.', '')

        # Quitar espacios
        number_str = number_str.replace(' ', '')

        # Convertir a float
        value = float(number_str)

        # Validar rango
        if 1_000_000 <= value <= 50_000_000:
            return value

    except (ValueError, AttributeError):
        pass

    return None

# Probar la función con tus ejemplos
print("🧪 Probando limpieza de salarios:\n")

ejemplos = [
    '$ 2.000.000,00 (Mensual)',
    '$ 1.800.000,00 (Mensual) + Comisiones',
    'Salario confidencial',
    '$1,5 a $2 millones',
    '$1.000.001 - $1.500.000',
    'A Convenir',
    '$3,5 a $4 millones',
    '8500000',
    '$3.500.000'
]

for ej in ejemplos:
    resultado = limpiar_salario_avanzado(ej)
    if resultado:
        print(f"  '{ej}' → ${resultado:,.0f}")
    else:
        print(f"  '{ej}' → None (excluido)")

print("\n✅ Función de limpieza lista")

🧪 Probando limpieza de salarios:

  '$ 2.000.000,00 (Mensual)' → $2,000,000
  '$ 1.800.000,00 (Mensual) + Comisiones' → $1,800,000
  'Salario confidencial' → None (excluido)
  '$1,5 a $2 millones' → $1,750,000
  '$1.000.001 - $1.500.000' → $1,250,000
  'A Convenir' → None (excluido)
  '$3,5 a $4 millones' → $3,750,000
  '8500000' → $8,500,000
  '$3.500.000' → $3,500,000

✅ Función de limpieza lista


In [15]:
# ========================================
# EXTRACCIÓN MEJORADA DE CARACTERÍSTICAS
# ========================================

def extraer_features_mejorado(titulo, descripcion):
    """
    Extrae más características para mejorar el R²
    """
    texto = f"{titulo} {descripcion}".lower()
    features = {}

    # 1. TECNOLOGÍAS (ampliadas)
    tecnologias = {
        'python': ['python', 'django', 'flask', 'fastapi'],
        'java': ['java', 'spring', 'hibernate'],
        'javascript': ['javascript', 'js', 'node', 'nodejs'],
        'react': ['react', 'reactjs', 'react.js'],
        'angular': ['angular', 'angularjs'],
        'vue': ['vue', 'vuejs'],
        'typescript': ['typescript', 'ts'],
        'sql': ['sql', 'mysql', 'postgresql', 'postgres', 'oracle'],
        'nosql': ['mongodb', 'mongo', 'redis', 'cassandra'],
        'aws': ['aws', 'amazon web'],
        'azure': ['azure', 'microsoft cloud'],
        'gcp': ['gcp', 'google cloud'],
        'docker': ['docker', 'container'],
        'kubernetes': ['kubernetes', 'k8s'],
        'dotnet': ['.net', 'dotnet', 'c#', 'csharp'],
        'php': ['php', 'laravel', 'symfony'],
        'ruby': ['ruby', 'rails'],
        'golang': ['golang', 'go '],
    }

    for tech, keywords in tecnologias.items():
        features[f'tech_{tech}'] = 1 if any(kw in texto for kw in keywords) else 0

    # Contador de tecnologías
    features['num_tecnologias'] = sum(1 for v in features.values() if v == 1)

    # 2. NIVEL DE INGLÉS (detallado)
    if any(w in texto for w in ['c2', 'c1', 'advanced', 'fluent', 'native', 'bilingue', 'bilingual']):
        features['ingles_nivel'] = 3  # Avanzado
    elif any(w in texto for w in ['b2', 'b1', 'intermediate', 'intermedio']):
        features['ingles_nivel'] = 2  # Intermedio
    elif any(w in texto for w in ['a2', 'a1', 'basic', 'basico', 'básico']):
        features['ingles_nivel'] = 1  # Básico
    else:
        features['ingles_nivel'] = 0  # Sin inglés

    # 3. SENIORITY (numérico)
    if any(w in texto for w in ['lead', 'principal', 'staff', 'arquitecto', 'architect']):
        features['seniority_nivel'] = 4
    elif any(w in texto for w in ['senior', 'sr.', 'sr ', 'ssr+']):
        features['seniority_nivel'] = 3
    elif any(w in texto for w in ['semi-senior', 'semisenior', 'ssr', 'mid-level', 'semi senior']):
        features['seniority_nivel'] = 2
    elif any(w in texto for w in ['junior', 'jr.', 'jr ', 'trainee', 'entry']):
        features['seniority_nivel'] = 1
    else:
        features['seniority_nivel'] = 0

    # 4. AÑOS DE EXPERIENCIA (extraer números)
    experiencia_matches = re.findall(r'(\d+)\s*(?:años?|years?)', texto)
    if experiencia_matches:
        features['anos_experiencia'] = int(experiencia_matches[0])
    else:
        # Inferir de seniority
        if features['seniority_nivel'] == 4:
            features['anos_experiencia'] = 8
        elif features['seniority_nivel'] == 3:
            features['anos_experiencia'] = 5
        elif features['seniority_nivel'] == 2:
            features['anos_experiencia'] = 3
        elif features['seniority_nivel'] == 1:
            features['anos_experiencia'] = 1
        else:
            features['anos_experiencia'] = 0

    # 5. MODALIDAD
    if any(w in texto for w in ['remoto', 'remote', 'wfh', 'work from home']):
        features['modalidad'] = 2  # Remoto
    elif any(w in texto for w in ['hibrido', 'híbrido', 'hybrid', 'mixto']):
        features['modalidad'] = 1  # Híbrido
    else:
        features['modalidad'] = 0  # Presencial

    # 6. CIUDAD (peso salarial)
    if any(w in texto for w in ['bogota', 'bogotá']):
        features['ciudad_peso'] = 3
    elif any(w in texto for w in ['medellin', 'medellín', 'cali']):
        features['ciudad_peso'] = 2
    elif any(w in texto for w in ['barranquilla', 'cartagena', 'bucaramanga']):
        features['ciudad_peso'] = 1
    else:
        features['ciudad_peso'] = 0

    # 7. TIPO DE DESARROLLO
    features['es_fullstack'] = 1 if any(w in texto for w in ['full stack', 'fullstack', 'full-stack']) else 0
    features['es_backend'] = 1 if 'backend' in texto or 'back-end' in texto else 0
    features['es_frontend'] = 1 if 'frontend' in texto or 'front-end' in texto else 0
    features['es_mobile'] = 1 if any(w in texto for w in ['mobile', 'android', 'ios', 'react native']) else 0
    features['es_devops'] = 1 if 'devops' in texto else 0
    features['es_data'] = 1 if any(w in texto for w in ['data', 'analytics', 'scientist', 'engineer']) else 0

    # 8. LONGITUDES
    features['titulo_length'] = len(titulo)
    features['descripcion_length'] = len(descripcion)

    return features

print("✅ Función de extracción mejorada lista")

✅ Función de extracción mejorada lista


In [16]:
# ========================================
# PROCESAMIENTO Y ENTRENAMIENTO
# ========================================

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.preprocessing import StandardScaler

print("🔧 Limpiando salarios con función mejorada...\n")

# Limpiar salarios
df['salario_limpio'] = df[COLUMN_SALARIO].apply(limpiar_salario_avanzado)

# Mostrar estadísticas de limpieza
total_original = len(df)
total_validos = df['salario_limpio'].notna().sum()
porcentaje = (total_validos / total_original) * 100

print(f"   Total registros: {total_original}")
print(f"   Salarios válidos: {total_validos} ({porcentaje:.1f}%)")
print(f"   Excluidos: {total_original - total_validos}\n")

# Mostrar algunos ejemplos de limpieza
print("📊 Ejemplos de limpieza:")
muestra = df[[COLUMN_SALARIO, 'salario_limpio']].head(10)
for idx, row in muestra.iterrows():
    original = row[COLUMN_SALARIO]
    limpio = row['salario_limpio']
    if pd.notna(limpio):
        print(f"   '{original}' → ${limpio:,.0f}")
    else:
        print(f"   '{original}' → Excluido")

# Filtrar datos válidos
df_clean = df.dropna(subset=['salario_limpio']).copy()

if len(df_clean) < 10:
    print(f"\n❌ ERROR: Muy pocos datos válidos ({len(df_clean)})")
else:
    # Remover outliers
    Q1 = df_clean['salario_limpio'].quantile(0.25)
    Q3 = df_clean['salario_limpio'].quantile(0.75)
    IQR = Q3 - Q1

    df_clean = df_clean[
        (df_clean['salario_limpio'] >= Q1 - 1.5 * IQR) &
        (df_clean['salario_limpio'] <= Q3 + 1.5 * IQR)
    ]

    print(f"\n   Después de remover outliers: {len(df_clean)}")
    print(f"   Rango: ${df_clean['salario_limpio'].min():,.0f} - ${df_clean['salario_limpio'].max():,.0f}\n")

    # Extraer características mejoradas
    print("🔧 Extrayendo características mejoradas...")

    features_list = []
    for _, row in df_clean.iterrows():
        features = extraer_features_mejorado(
            str(row[COLUMN_TITULO]),
            str(row[COLUMN_DESCRIPCION])
        )
        features_list.append(features)

    X = pd.DataFrame(features_list)
    y = df_clean['salario_limpio'].reset_index(drop=True)

    print(f"   ✓ {len(X.columns)} características extraídas")
    print(f"\nCaracterísticas principales:")
    for col in X.columns[:10]:
        print(f"   - {col}")

    # Entrenar con validación
    print(f"\n🚀 Entrenando modelo mejorado...\n")

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Escalar features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Entrenar modelo
    model = GradientBoostingRegressor(
        n_estimators=300,
        max_depth=7,
        learning_rate=0.05,
        subsample=0.8,
        min_samples_split=10,
        min_samples_leaf=5,
        random_state=42
    )

    model.fit(X_train_scaled, y_train)

    # Evaluar
    train_score = model.score(X_train_scaled, y_train)
    test_score = model.score(X_test_scaled, y_test)

    y_pred = model.predict(X_test_scaled)
    mae = np.mean(np.abs(y_test - y_pred))
    rmse = np.sqrt(np.mean((y_test - y_pred) ** 2))
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

    print("✅ Modelo entrenado\n")
    print("=" * 60)
    print("📊 MÉTRICAS DEL MODELO")
    print("=" * 60)
    print(f"\n📈 R² Score:")
    print(f"   Train: {train_score:.4f}")
    print(f"   Test:  {test_score:.4f}")

    if test_score < 0:
        print(f"\n   ⚠️ R² negativo indica que el modelo no generaliza bien")
        print(f"   Posibles causas:")
        print(f"   - Pocos datos de entrenamiento")
        print(f"   - Datos muy diversos/ruidosos")
        print(f"   - Necesita más features relevantes")

    print(f"\n💰 Errores:")
    print(f"   MAE:  ${mae:,.0f} COP")
    print(f"   RMSE: ${rmse:,.0f} COP")
    print(f"   MAPE: {mape:.1f}%")

    print(f"\n📊 Estadísticas:")
    print(f"   Promedio: ${y.mean():,.0f} COP")
    print(f"   Mediana:  ${y.median():,.0f} COP")
    print(f"   Desv. Est: ${y.std():,.0f} COP")

    # Top features importantes
    print(f"\n🔑 Top 10 características más importantes:")
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)

    for idx, row in feature_importance.head(10).iterrows():
        print(f"   {row['feature']}: {row['importance']:.4f}")

    print("\n" + "=" * 60)

🔧 Limpiando salarios con función mejorada...

   Total registros: 1451
   Salarios válidos: 848 (58.4%)
   Excluidos: 603

📊 Ejemplos de limpieza:
   '$ 8.400.000,00 (Mensual)' → $8,400,000
   '$ 6.000.000,00 (Mensual)' → $6,000,000
   '$ 2.500.000,00 (Mensual)' → $2,500,000
   '$ 5.500.000,00 (Mensual)' → $5,500,000
   '$ 9.500.000,00 (Mensual)' → $9,500,000
   '$ 3.000.000,00 (Mensual)' → $3,000,000
   '$ 2.700.000,00 (Mensual)' → $2,700,000
   '$ 5.000.000,00 (Mensual)' → $5,000,000
   '$ 2.000.000,00 (Mensual)' → $2,000,000
   '$ 4.000.000,00 (Mensual)' → $4,000,000

   Después de remover outliers: 820
   Rango: $1,250,000 - $10,000,000

🔧 Extrayendo características mejoradas...
   ✓ 32 características extraídas

Características principales:
   - tech_python
   - tech_java
   - tech_javascript
   - tech_react
   - tech_angular
   - tech_vue
   - tech_typescript
   - tech_sql
   - tech_nosql
   - tech_aws

🚀 Entrenando modelo mejorado...

✅ Modelo entrenado

📊 MÉTRICAS DEL MODELO

📈

In [17]:
# ========================================
# PREDICCIONES
# ========================================

def predecir_salario(titulo, descripcion):
    """Predice el salario para una descripción"""
    features = extraer_features_mejorado(titulo, descripcion)
    X_pred = pd.DataFrame([features], columns=X.columns)
    X_pred_scaled = scaler.transform(X_pred)

    salario = model.predict(X_pred_scaled)[0]
    lower = salario - 1.96 * rmse
    upper = salario + 1.96 * rmse

    return {
        'salario': salario,
        'lower': max(1_000_000, lower),
        'upper': min(50_000_000, upper),
        'features': features
    }

# Ejemplo
print("\n" + "=" * 80)
print("🔮 EJEMPLO DE PREDICCIÓN")
print("=" * 80)

ejemplo_titulo = "Desarrollador Full Stack Senior"
ejemplo_desc = """
Requisitos:
- 5 años de experiencia en desarrollo web
- Python, Django, React, TypeScript
- AWS, Docker, Kubernetes
- PostgreSQL, MongoDB
- Inglés intermedio (B2)
- Modalidad remota 100%
Ubicación: Bogotá, Colombia
"""

pred = predecir_salario(ejemplo_titulo, ejemplo_desc)

print(f"\nTítulo: {ejemplo_titulo}")
print(f"\n💰 SALARIO ESTIMADO: ${pred['salario']:,.0f} COP")
print(f"\n📊 Rango de confianza (95%):")
print(f"   Mínimo: ${pred['lower']:,.0f} COP")
print(f"   Máximo: ${pred['upper']:,.0f} COP")

print(f"\n🔍 Características detectadas:")
print(f"   Nivel de inglés: {pred['features']['ingles_nivel']}")
print(f"   Seniority: {pred['features']['seniority_nivel']}")
print(f"   Años experiencia: {pred['features']['anos_experiencia']}")
print(f"   Tecnologías: {pred['features']['num_tecnologias']}")
print(f"   Modalidad: {['Presencial', 'Híbrido', 'Remoto'][pred['features']['modalidad']]}")

print("\n" + "=" * 80)


🔮 EJEMPLO DE PREDICCIÓN

Título: Desarrollador Full Stack Senior

💰 SALARIO ESTIMADO: $3,696,848 COP

📊 Rango de confianza (95%):
   Mínimo: $1,000,000 COP
   Máximo: $6,927,539 COP

🔍 Características detectadas:
   Nivel de inglés: 2
   Seniority: 3
   Años experiencia: 5
   Tecnologías: 8
   Modalidad: Presencial



In [10]:
# TUS PREDICCIONES
print("💡 Haz tu propia predicción\n")

mi_titulo = input("Título del puesto: ").strip() or "Desarrollador Python Junior"
mi_desc = input("Descripción (Enter para ejemplo): ").strip() or "Python Django. 1 año. Bogotá presencial. Sin inglés."

pred = predecir_salario(mi_titulo, mi_desc)

print("\n" + "=" * 80)
print(f"💰 Salario estimado: ${pred['salario']:,.0f} COP")
print(f"📊 Rango: ${pred['lower']:,.0f} - ${pred['upper']:,.0f} COP")
print("=" * 80)

💡 Haz tu propia predicción

Título del puesto: Angular desarrollador api rest
Descripción (Enter para ejemplo): api rest 2 años de experiencia

💰 Salario estimado: $3,463,920 COP
📊 Rango: $1,000,000 - $6,694,611 COP
